In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from huggingface_hub import login
login('')

/home/jovyan/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.data_utils import SIFT50MDataset
import os
from datasets import load_dataset, Dataset

sift_dataset = load_dataset(
    'amazon-agi/SIFT-50M',
    name='closed_ended_comparison',
    split='train',
    trust_remote_code=True
)
sift_dataset = sift_dataset.shuffle(seed=90)
df = sift_dataset.select(range(2000)).to_pandas()
print(len(sift_dataset))
allowed_values = ["common_voice_de", "common_voice_en",'vctk_en'] # "multilingual_librispeech_de" The datasets were taking too long to get donwloaded so I restricted it to certain subsets

filtered_df = df[df["data_source"].isin(allowed_values)]

# Count number of entries per value
counts = filtered_df["data_source"].value_counts()
print(counts)
sift_data = Dataset.from_pandas(filtered_df)
# Define the base datasets paths (replace with your actual paths)


3613791
data_source
common_voice_en    279
common_voice_de    114
vctk_en             13
Name: count, dtype: int64


In [ ]:
filtered_df.iloc[100].messages

In [ ]:
filtered_df_1 = filtered_df[filtered_df["data_source"].isin(['vctk_en'])]
print(filtered_df_1.iloc[1]['messages'])

In [4]:
total_len = len(sift_data)
eval_len = int(0.2*total_len)

eval_ds = sift_data.select(range(eval_len))
train_ds = sift_data.select(range(eval_len,total_len))


In [5]:
base_datasets_root = "/home/jovyan/.cache/huggingface/datasets"
base_datasets_paths = {
    "common_voice_de": None, # No longer needs a path, handled by load_dataset
    #"multilingual_librispeech_de": None, # No longer needs a path, handled by load_dataset
    "common_voice_en": None, # No longer needs a path, handled by load_dataset
    "vctk_en": "./vctk_corpus" # VCTK still needs a root path for torchaudio
}

# Create an instance of the SIFT50MDataset
sift_iterable_dataset_train = SIFT50MDataset(sift_dataset=train_ds, base_datasets_paths=base_datasets_paths)
sift_iterable_dataset_eval = SIFT50MDataset(sift_dataset=eval_ds, base_datasets_paths=base_datasets_paths)


common_voice_de None
common_voice_en None
vctk_en ./vctk_corpus
common_voice_de None
common_voice_en None
vctk_en ./vctk_corpus


In [6]:

%reload_ext autoreload
    
from src.train_qlora import train_model

train_model( eval_ds = sift_iterable_dataset_eval,train_ds = sift_iterable_dataset_train)

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 12483.05it/s]
`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 19065.02it/s]
/home/jovyan/.local/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/home/jovyan/.local/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 151645, 'pad_token_id': 151643}.


KeyError: 'Unexpected input keys in examples: [].'